In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

data_path = "boston.csv"
data = pd.read_csv(data_path)

missing_values = data.isnull().sum().sum()
print(f"Пропущені значення: {missing_values}")

data.dropna(inplace=True)

print("\n 3. Визначення ознак та цільової змінної...")
y = data["MEDV"].values  # Ціна житла
X = data.drop(columns=["MEDV"]).values  # Ознаки
print(f"Розмірність X: {X.shape}, Розмірність y: {y.shape}")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n 5. Розбиття даних...")
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
print(f"Навчальний набір: {X_train.shape}, Тестовий набір: {X_test.shape}")

X_single = X_scaled[:, np.newaxis, 5]  
X_train_single, X_test_single, _, _ = train_test_split(X_single, y, test_size=0.3, random_state=42)

lin_reg_single = LinearRegression()
lin_reg_single.fit(X_train_single, y_train)
y_pred_single = lin_reg_single.predict(X_test_single)

lin_reg_multi = LinearRegression()
lin_reg_multi.fit(X_train, y_train)
y_pred_multi = lin_reg_multi.predict(X_test)

print("\n 8. Оцінка моделей...")
mse_single = mean_squared_error(y_test, y_pred_single)
r2_single = r2_score(y_test, y_pred_single)

mse_multi = mean_squared_error(y_test, y_pred_multi)
r2_multi = r2_score(y_test, y_pred_multi)

print(f" Однозмінна регресія: MSE={mse_single:.4f}, R2={r2_single:.4f}")
print(f" Мультизмінна регресія: MSE={mse_multi:.4f}, R2={r2_multi:.4f}")
)
print("\n 9. Виконання K-Fold крос-валідації...")
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(lin_reg_multi, X_scaled, y, cv=kf, scoring='r2')
print(f"Середнє R2 за K-Fold: {np.mean(cross_val_scores):.4f}")


ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)


lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

print("\n 11. Оцінка оптимізованих моделей...")
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f" Ridge регресія: MSE={mse_ridge:.4f}, R2={r2_ridge:.4f}")
print(f"Lasso регресія: MSE={mse_lasso:.4f}, R2={r2_lasso:.4f}")


plt.figure(figsize=(8, 5))
sns.scatterplot(x=X_test_single.flatten(), y=y_test, label="Реальні значення", color="blue")
sns.lineplot(x=X_test_single.flatten(), y=y_pred_single, label="Прогноз", color="red")
plt.xlabel("Середня кількість кімнат (RM)")
plt.ylabel("Ціна житла (MEDV)")
plt.title("Лінійна регресія з однією змінною")
plt.legend()
plt.show()

plt.figure(figsize=(8, 5))
sns.scatterplot(x=y_test, y=y_pred_multi, label="Прогнозовані значення", color="red")
sns.lineplot(x=y_test, y=y_test, label="Ідеальна лінія", color="blue")
plt.xlabel("Реальні значення")
plt.ylabel("Прогнозовані значення")
plt.title("Мультизмінна лінійна регресія")
plt.legend()
plt.show()

print("\n 13. Вибір найкращої моделі на основі метрик...")

models = {
    "Однозмінна регресія": (mse_single, r2_single),
    "Мультизмінна регресія": (mse_multi, r2_multi),
    "Ridge регресія": (mse_ridge, r2_ridge),
    "Lasso регресія": (mse_lasso, r2_lasso),
}

best_model = max(models, key=lambda x: models[x][1])  
print(f"Найкраща модель: {best_model} (R2 = {models[best_model][1]:.4f})")
